In [11]:
import pandas as pd
import numpy as np
from Pokemonarena.Pokemon import Pokemon
from pokemon_factory import Pokemon_Factory
from Pokemonarena.fight import Fight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [2]:
df_pokemon = pd.read_csv('pokemon.csv')
df_mapping = pd.read_csv('pokemon_mapping.csv', encoding='cp1250', delimiter=';')

In [3]:
df_pokemon = df_pokemon.drop(['name', 'japanese_name'], axis = 1)
df_mapping = df_mapping.drop(['Englisch'], axis = 1)
df_mapping = df_mapping.dropna()
df_mapping['Id']=df_mapping['Id'].astype('int32')
df = df_pokemon.merge(df_mapping, how='inner', left_on=['pokedex_number'], right_on=['Id'])
df = df.rename(columns={"Deutsch": "name", 'sp_attack' : 'specialAttack' , 'sp_defense' : 'specialDefense'})

In [4]:
columns = ['name', 'hp', 'attack' , 'defense' , 'specialAttack','specialDefense', 'speed']
factory = Pokemon_Factory(df,columns)

In [5]:
factory = Pokemon_Factory(df,columns)
pokelist = factory.create_Pokemon_Sample(40)

In [6]:
dfs = []
for i in range(0, len(pokelist)):
    for j in range(0, len(pokelist)):
        if j != i:
            fight = Fight(pokelist[i], pokelist[j])
            dfs.append(pd.DataFrame.from_dict(fight.fight_ml()))

In [7]:
fight_df = pd.concat(dfs, axis=0)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = fight_df.drop(['Gewonnen', 'name1', 'name2'], axis = 1)
y = fight_df['Gewonnen']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
model = LogisticRegression()
model.fit(X_train,y_train)
predictions = model.predict(X_test)
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.9320388349514563
[[229  19]
 [ 16 251]]
              precision    recall  f1-score   support

       False       0.93      0.92      0.93       248
        True       0.93      0.94      0.93       267

    accuracy                           0.93       515
   macro avg       0.93      0.93      0.93       515
weighted avg       0.93      0.93      0.93       515

